In [1]:
!pip install ruamel.yaml

In [7]:
from ruamel.yaml import YAML
from itertools import repeat

In [53]:
session_configuration = """\
session:
    name: nominal test
    scenarios:
        - name: Sc1
          weight: 2
          steps:
                loop:
                    tasks:
                        - name: step 1
                          duration: 10
                          resources : 50   
                          wait: 30
        - name: Sc2
          weight: 10
          steps:
                init:
                    tasks:
                        - name: load data
                          duration: 5
                          resources : 3
                          parallel: 10
                        - name: verify data
                          duration: 10
                loop:
                    repeat: 2
                    tasks:
                        - name: step 1-3
                          duration: 2
                          resources : 5   
                          wait: 5
                          repeat: 3
                        - name: step 7
                          duration: 5
                          resources : 10      
                          wait: 30
                finally:
                    tasks:
                        - name: whole step
                          duration: 3
                          resources : 7
                          repeat: 4       

"""



In [54]:
yaml = YAML(typ='safe')   # default, if not specfied, is 'rt' (round-trip)

scenarios = yaml.load(session_configuration)


In [5]:
print(f"session name: {scenarios['session']['name']}")

session name: nominal test


In [ ]:
# scanario 

In [55]:
scenarios_pool = scenarios['session']['scenarios']
#print(scenarios_pool)
pool = []
for i in range(len(scenarios_pool)):
    weight = scenarios_pool[i]['weight']
    pool.extend(repeat(i, weight))

print(f"pool: {pool}")

pool: [0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [60]:
def run_step_tasks(tasks):
    print(f"run_step_tasks::tasks: {tasks}")
    for task in tasks:
        #run_task(task)
        print(f"run_step_tasks::task: {task}")

In [61]:
def run_scenario(index):
    scenario = scenarios_pool[index]
    print("scenario: {scenario}")
    print(f"name: {scenario['name']}")
    print(f"steps: {scenario['steps']}")
    if 'init' in scenario['steps']:
        print("has init") 
        print("run_step_tasks init")
        run_step_tasks(scenario['steps']['init']['tasks'])
    if 'loop' in scenario['steps']:
        print("has loop")
        step_loop = scenario['steps']['loop']
        if 'repeat' in step_loop:
            counter = 0
            while counter < step_loop['repeat']:
                print("run_step_tasks loop")
                run_step_tasks(scenario['steps']['loop']['tasks'])
                counter += 1
        else:
            print("run_step_tasks loop infinite")
            run_step_tasks(scenario['steps']['loop']['tasks'])
    if 'finally' in scenario['steps']:
        print("has finally")
        print("run_step_tasks finally")
        run_step_tasks(scenario['steps']['finally']['tasks'])

        

In [62]:
run_scenario(0)

scenario: {scenario}
name: Sc1
steps: {'loop': {'tasks': [{'name': 'step 1', 'duration': 10, 'resources': 50, 'wait': 30}]}}
has loop
run_step_tasks loop infinite
run_step_tasks::tasks: [{'name': 'step 1', 'duration': 10, 'resources': 50, 'wait': 30}]
run_step_tasks::task: {'name': 'step 1', 'duration': 10, 'resources': 50, 'wait': 30}


In [63]:
run_scenario(1)

scenario: {scenario}
name: Sc2
steps: {'init': {'tasks': [{'name': 'load data', 'duration': 5, 'resources': 3, 'parallel': 10}, {'name': 'verify data', 'duration': 10}]}, 'loop': {'repeat': 2, 'tasks': [{'name': 'step 1-3', 'duration': 2, 'resources': 5, 'wait': 5, 'repeat': 3}, {'name': 'step 7', 'duration': 5, 'resources': 10, 'wait': 30}]}, 'finally': {'tasks': [{'name': 'whole step', 'duration': 3, 'resources': 7, 'repeat': 4}]}}
has init
run_step_tasks init
run_step_tasks::tasks: [{'name': 'load data', 'duration': 5, 'resources': 3, 'parallel': 10}, {'name': 'verify data', 'duration': 10}]
run_step_tasks::task: {'name': 'load data', 'duration': 5, 'resources': 3, 'parallel': 10}
run_step_tasks::task: {'name': 'verify data', 'duration': 10}
has loop
run_step_tasks loop
run_step_tasks::tasks: [{'name': 'step 1-3', 'duration': 2, 'resources': 5, 'wait': 5, 'repeat': 3}, {'name': 'step 7', 'duration': 5, 'resources': 10, 'wait': 30}]
run_step_tasks::task: {'name': 'step 1-3', 'duratio

In [ ]:
# task 

In [74]:
def run_task(task):
    print(f"run_task::task: {task}")
    max_count = 1
    if 'repeat' in task:
        max_count = task['repeat']
    counter = 0
    while counter < max_count:
        print(f"run_task::running task {task['name']} for {task['duration']}")
        if 'resources' in task:
            res_amount = task['resources']
            if 'parallel' in task:
                print(f"run_task::in parallel")
                res_amount = res_amount * task['parallel']
            print(f"run_task::resources amount {res_amount}")
        if 'wait' in task:
            print(f"run_task::then wait for {task['wait']}")
        counter += 1

In [75]:
# basic 1 task and wait
run_task(scenarios['session']['scenarios'][0]['steps']['loop']['tasks'][0])

run_task::task: {'name': 'step 1', 'duration': 10, 'resources': 50, 'wait': 30}
run_task::running task step 1 for 10
run_task::resources amount 50
run_task::then wait for 30


In [76]:
# parallel task
# should multiply ammount
run_task(scenarios['session']['scenarios'][1]['steps']['init']['tasks'][0])

run_task::task: {'name': 'load data', 'duration': 5, 'resources': 3, 'parallel': 10}
run_task::running task load data for 5
run_task::in parallel
run_task::resources amount 30


In [77]:
# no res task (verify ...)
run_task(scenarios['session']['scenarios'][1]['steps']['init']['tasks'][1])

run_task::task: {'name': 'verify data', 'duration': 10}
run_task::running task verify data for 10


In [78]:
# task with wait
run_task(scenarios['session']['scenarios'][1]['steps']['loop']['tasks'][1])

run_task::task: {'name': 'step 7', 'duration': 5, 'resources': 10, 'wait': 30}
run_task::running task step 7 for 5
run_task::resources amount 10
run_task::then wait for 30


In [79]:
# task with repeat (no wait between repeats)
run_task(scenarios['session']['scenarios'][1]['steps']['finally']['tasks'][0])

run_task::task: {'name': 'whole step', 'duration': 3, 'resources': 7, 'repeat': 4}
run_task::running task whole step for 3
run_task::resources amount 7
run_task::running task whole step for 3
run_task::resources amount 7
run_task::running task whole step for 3
run_task::resources amount 7
run_task::running task whole step for 3
run_task::resources amount 7


In [80]:
# task with repeat (wait between repeats)
run_task(scenarios['session']['scenarios'][1]['steps']['loop']['tasks'][0])

run_task::task: {'name': 'step 1-3', 'duration': 2, 'resources': 5, 'wait': 5, 'repeat': 3}
run_task::running task step 1-3 for 2
run_task::resources amount 5
run_task::then wait for 5
run_task::running task step 1-3 for 2
run_task::resources amount 5
run_task::then wait for 5
run_task::running task step 1-3 for 2
run_task::resources amount 5
run_task::then wait for 5
